In [ ]:
# Basic imports
import numpy as np
import pandas as pd

# Plotting tools
import bokeh
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, ColumnDataSource, PrintfTickFormatter, HoverTool
from bokeh.transform import transform
from bokeh.palettes import Viridis256

import cellseg.plot

bokeh.io.output_notebook()

## Figure plotting
The purpose of this code is to generate the figure panels for publication from the data collected by `in_vitro_cell_segmentation.ipynb`

We can start by importing the data that we want to work with, merging the dataframes to connect the condition with the data, and sorting the dataframe:

In [ ]:
# Use read_csv to create a dataframe from the data csv 
df_data = pd.read_csv('data/quantification/20220831-data.csv', comment='#')

In [ ]:
# Use read_csv to create a dataframe from the look up table
df_meta = pd.read_csv('data/quantification/20220915_data_lookup_table.csv', comment='#')

In [ ]:
# Use merge the two dataframes, using round, plate, and well to combine them
df = pd.merge(df_data, df_meta, on=["Round", "Plate", "Well"])

In [ ]:
#We can sort the values to ensure they're in a logical order
df = df.sort_values(by = ['Round', 'Plate', 'Well'], axis = 0)

# Drop any empty rows and reset the index so that it is in sequence
df = df.dropna().reset_index(drop=True)

We can perform some initial analysis on the data to determine the percent positive area of each well, the scaled area (to enable plotting at a smaller size), and the total brightness per signal area.

In [ ]:
# Determine the percent positive and add it to a new column
df['Percent Positive'] = df['Signal Area']*100/df['Bright Field Area']
df['Percent Positive Scaled'] = df['Percent Positive']*0.5
df['Total Brightness per Signal Area'] = df['Total Brightness']/df['Signal Area']

We can then examine the resulting dataframe: 

In [ ]:
# Create a index to only include valid data (selected by experimenter)
inds = df['Include'] == True

# Create a reduced data frame of this data
df_reduced = df.loc[inds]

# Look at the dataframe 
df_reduced.head()

Now that we have our data, we want to generate the figures:

## Extended Data Figure 2

First, we want to compare PHP.eB against AAV9 at a range of doses without a receptor.

In [ ]:
# We want to make a new dataframe with just the Ly6a and receptor-free conditions.
# First, we can find the indices of these receptors.
inds = (df_reduced['Receptor'] == 'Ly6a') | (df_reduced['Receptor'] == 'None')

# Make this a new dataframe df_edf2
df_edf2 = df_reduced.loc[inds]

In [ ]:
# Next, we can reduce this to just the PHP.eB and AAV9 conditions.
inds = (df_edf2['Virus'] == 'PHP.eB') | (df_edf2['Virus'] == 'AAV9')

df_edf2 = df_edf2.loc[inds]

In [ ]:
# We only want to examine a single round, since including data from other rounds could skew the results due to variability. 
inds = (df_edf2['Round'] == 10)

df_edf2 = df_edf2[inds]

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_edf2 = df_edf2.astype({"Dose": str})

In [ ]:
# We can determine the mean of each of the values
df_edf2 = df_edf2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
# Setting our index to only examine data with Receptor none, we can plot the untransfected AAV9 and PHP.eB transfections
inds = (df_edf2['Receptor'] == 'None')

# Dataframe to use
source = ColumnDataSource(df_edf2[inds])

# Plotting parameters
p = figure(x_range = df_edf2['Virus'].unique(), 
           y_range = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'],
           title = 'No Receptor',
           y_axis_label = 'Dose (vg/well)',
           x_axis_label = 'Virus',
          width = 400,
          height = 500)

# Set the color map based on the median cell brightness in df_edf2
color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf2['Total Brightness per Signal Area'].min(), high = df_edf2['Total Brightness per Signal Area'].max())

# Add a color bar
color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

# Plot the percent positive scaled data 0.5*percent positive 
p.scatter(x = 'Virus', y = 'Dose', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
# Setting our index to only examine data with Receptor Ly6a, we can plot the Ly6a transfected AAV9 and PHP.eB transfections
inds = (df_edf2['Receptor'] == 'Ly6a')

source = ColumnDataSource(df_edf2[inds])

p = figure(x_range = df_edf2['Virus'].unique(), 
           y_range = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'],
           title = 'Transfected with Ly6a',
           y_axis_label = 'Dose (vg/well)',
           x_axis_label = 'Virus',
          width = 400,
          height = 500)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf2['Total Brightness per Signal Area'].min(), high = df_edf2['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Dose', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_edf2['Percent Positive']), max(df_edf2.loc[inds]['Percent Positive']), df_edf2['Total Brightness per Signal Area'].min(), df_edf2['Total Brightness per Signal Area'].max()

## Extended Data Figure 5

In [ ]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf5
df_edf5 = df_reduced.loc[inds]

In [ ]:
# Next, we can reduce this to just to round 7.
inds = (((df_edf5['Virus'] != 'PHP.C1') & (df_edf5['Virus'] != 'PHP.C3') & (df_edf5['Round'] == 7)) | ((df_edf5['Virus'] == 'PHP.C1') | (df_edf5['Virus'] == 'PHP.C3')) & (df_edf5['Round'] < 8))

df_edf5 = df_edf5.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_edf5 = df_edf5.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_edf5)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      '9P08',
                      '9P16',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F',
                      '9P13',
                      '9P33',
                      '9P39'],
          y_range = ['Cd59',
                     'Lynx1',
                     'Lypd5',
                     'Ly6h',
                     'Ly6e', 
                     'Ly6c1',
                     'Ly6a'],
           title = 'Receptor screen of Ly6 family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf5['Total Brightness per Signal Area'].min(), high = df_edf5['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
inds = (df_edf5['Virus'] != 'PHP.C1') & (df_edf5['Virus'] != 'PHP.C3')
min(df_edf5.loc[inds]['Percent Positive']), max(df_edf5.loc[inds]['Percent Positive']), df_edf5['Total Brightness per Signal Area'].min(), df_edf5['Total Brightness per Signal Area'].max()

## Extended Data Figure 6

In [ ]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf6
df_edf6 = df_reduced.loc[inds]

In [ ]:
inds = (df_edf6['Round'] == 10)

df_edf6 = df_edf6.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_edf6 = df_edf6.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_edf6)

p = figure(x_range = ['9P31',
                      '9P36'],
          y_range = ['Car15',
                     'Car14',
                     'Car12',
                     'Car9',
                     'Car4'],
           title = 'Receptor screen of carbonic anhydrase family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width = 400, 
           height = 500)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf6['Total Brightness per Signal Area'].min(), high = df_edf6['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
inds = (df_edf6['Receptor'] == 'Car4') | (df_edf6['Receptor'] == 'Car9') | (df_edf6['Receptor'] == 'Car12') | (df_edf6['Receptor'] == 'Car14') | (df_edf6['Receptor'] == 'Car15')
min(df_edf6.loc[inds]['Percent Positive']), max(df_edf6.loc[inds]['Percent Positive']), df_edf6['Total Brightness per Signal Area'].min(), df_edf6['Total Brightness per Signal Area'].max()

## Extended Data Figure 8

In [ ]:
#We can sort the values to ensure they're in a logical order
inds = (df_reduced['Dose'] == 1e9)

df_edf8 = df_reduced.loc[inds]

In [ ]:
# Next, we can reduce this to just to round 7.
inds = (((df_edf8['Virus'] != 'PHP.C1') & (df_edf8['Virus'] != 'PHP.C3') & (df_edf8['Virus'] != '9P13') & (df_edf8['Virus'] != '9P33')) & (df_edf8['Round'] < 8)) | \
        ((df_edf8['Virus'] == 'PHP.C1') | (df_edf8['Virus'] == 'PHP.C3')) & (df_edf8['Round'] < 7)

df_edf8 = df_edf8.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_edf8 = df_edf8.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_edf8)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F'],
                      #'9P31',
                      #'9P36'],
          y_range = ['Tek',
                     'Slco1c1',
                     'Slco1a4',
                     'Slc7a1',
                     'Slc6a6',
                     'Slc2a1', 
                     'Slc22a8',
                     'Serinc3',
                     'Prom1',
                     'Podxl',
                     'Pecam1', 
                     'Paqr5', 
                     'Ocln', 
                     'Ly6e', 
                     'Ly6c1', 
                     'Ly6a', 
                     'Kitl', 
                     'Kdr', 
                     'Itm2b', 
                     'Igf1r', 
                     'Ifitm3', 
                     'Ifitm2', 
                     'Flt1', 
                     'Fcgrt', 
                     'Esam', 
                     'Eng',
                     'Clic4', 
                     'Clic1', 
                     'Clec2d', 
                     'Cldn5', 
                     'Cdh5', 
                     'Cd34', 
                     'Car4', 
                     'Bsg', 
                     'App', 
                     'Adgrl4',
                     'Adgrf5', 
                     'Acvrl1', 
                     'Abcg2', 
                     'Abcb1a'],
           title = 'Candidate Receptors',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=1000)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf8['Total Brightness per Signal Area'].min(), high = df_edf8['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_edf8['Percent Positive']), max(df_edf8['Percent Positive']), df_edf8['Total Brightness per Signal Area'].min(), df_edf8['Total Brightness per Signal Area'].max()

In [ ]:
inds = (df_reduced['Dose'] == 1e9)

df_edf8_2 = df_reduced.loc[inds]

In [ ]:
# Next, we can reduce this to just to round 7.
inds = ((df_edf8_2['Virus'] == '9P31') | (df_edf8_2['Virus'] == '9P36')) & ((df_edf8_2['Round'] > 6) & (df_edf8_2['Round'] < 10))

df_edf8_2 = df_edf8_2.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_edf8_2 = df_edf8_2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_edf8_2)

p = figure(x_range = ['9P31',
                      '9P36'],
          y_range = ['Tek',
                     'Sclo1c1',
                     'Slco1a4',
                     'Slc7a1',
                     'Slc6a6',
                     'Slc2a1', 
                     'Slc22a8',
                     'Serinc3',
                     'Prom1',
                     'Podxl',
                     'Pecam1', 
                     'Paqr5', 
                     'Ocln', 
                     'Ly6e', 
                     'Ly6c1', 
                     'Ly6a', 
                     'Kitl', 
                     'Kdr', 
                     'Itm2b', 
                     'Igf1r', 
                     'Ifitm3', 
                     'Ifitm2', 
                     'Flt1', 
                     'Fcgrt', 
                     'Esam', 
                     'Eng',
                     'Clic4', 
                     'Clic1', 
                     'Clec2d', 
                     'Cldn5', 
                     'Cdh5', 
                     'Cd34', 
                     'Car4', 
                     'Bsg', 
                     'App', 
                     'Adgrl4',
                     'Adgrf5', 
                     'Acvrl1', 
                     'Abcg2', 
                     'Abcb1a'],
           title = 'Candidate Receptors',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=300, 
           height=1000)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf8['Total Brightness per Signal Area'].min(), high = df_edf8['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
inds = (df_edf8_2['Virus'] != 'PHP.C1') & (df_edf8_2['Virus'] != 'PHP.C3')
min(df_edf8_2.loc[inds]['Percent Positive']), max(df_edf8_2.loc[inds]['Percent Positive']), min(df_edf8_2.loc[inds]['Total Brightness per Signal Area']), max(df_edf8_2.loc[inds]['Total Brightness per Signal Area'])

## Extended Data Figure 10

In [ ]:
# Select the round for the corresponding figure
inds = (df_reduced['Round'] == 11) & (df_reduced['Dose'] != 5e8)

# Make this a new dataframe df_edf10
df_edf10 = df_reduced.loc[inds]

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_edf10 = df_edf10.astype({"Dose": str})

In [ ]:
# We can determine the mean of each of the values
df_edf10 = df_edf10.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_edf10

In [ ]:
for virus in ['PHP.eB', 'PHP.C2', 'AAV-F', 'eC 1', 'eC 3', 'eC 4', 'eC 5', 'eC 8','eC 9', 'eC 11', 'eC 12', 'eC 14', 'eC 15', 'eC 16', 'eC 19']:
    inds = (df_edf10['Virus'] == virus)
    
    source = ColumnDataSource(df_edf10[inds])

    p = figure(x_range = ['None',
                         'Ly6c1'],
              y_range = ['50000000.0',
                         '100000000.0',
                         '1000000000.0'],
               title = virus,
               x_axis_label = 'Receptor',
               y_axis_label = 'Dose',
               width=400, 
               height=400)

    color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf10['Total Brightness per Signal Area'].min(), high = df_edf10['Total Brightness per Signal Area'].max())

    color_bar = ColorBar(color_mapper = color_mapper,
                         location = (0, 0),
                         ticker = BasicTicker())

    p.add_layout(color_bar, 'right')

    p.scatter(x = 'Receptor', y = 'Dose', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

    p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

    p.xaxis.major_label_orientation = np.pi/2

    #p.output_backend = "svg"

    show(p)

In [ ]:
min(df_edf10['Percent Positive']), max(df_edf10['Percent Positive']), min(df_edf10['Total Brightness per Signal Area']), max(df_edf10['Total Brightness per Signal Area'])

## Figure 4

### Ly6a mutated

In [ ]:
inds = (df_reduced['Virus'] == 'PHP.eB') & (df_reduced['Round'] == 12) & (df_reduced['Dose'] != 1e10) 

df_4_pt1 = df_reduced.loc[inds]

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt1 = df_4_pt1.astype({"Dose": str})

In [ ]:
# We can determine the mean of each of the values
df_4_pt1 = df_4_pt1.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_4_pt1

In [ ]:
source = ColumnDataSource(df_4_pt1)

p = figure(x_range = ['10000000.0',
                      '100000000.0',
                      '1000000000.0'],
          y_range = ['Ly6a A58R',
                     'Ly6a'],
           title = 'Mutated Ly6a Infectivity Assay',
           x_axis_label = 'Dose',
           y_axis_label = 'Receptor',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt1['Total Brightness per Signal Area'].min(), high = df_4_pt1['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Dose', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_4_pt1['Percent Positive']), max(df_4_pt1['Percent Positive']), min(df_4_pt1['Total Brightness per Signal Area']), max(df_4_pt1['Total Brightness per Signal Area'])

In [ ]:
inds = (df_reduced['Virus'] == '9P31') & (df_reduced['Round'] == 12) & (df_reduced['Dose'] != 1e10) 

df_4_pt2 = df_reduced.loc[inds]

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt2 = df_4_pt2.astype({"Dose": str})

In [ ]:
# We can determine the mean of each of the values
df_4_pt2 = df_4_pt2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_4_pt2

In [ ]:
source = ColumnDataSource(df_4_pt2)

p = figure(x_range = ['10000000.0',
                      '100000000.0',
                      '1000000000.0'],
          y_range = ['msCar4 L71R',
                     'msCar4'],
           title = 'Mutated Car4 Infectivity Assay',
           x_axis_label = 'Dose',
           y_axis_label = 'Receptor',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt2['Total Brightness per Signal Area'].min(), high = df_4_pt2['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Dose', y = 'Receptor', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_4_pt2['Percent Positive']), max(df_4_pt2['Percent Positive']), min(df_4_pt2['Total Brightness per Signal Area']), max(df_4_pt2['Total Brightness per Signal Area'])

In [ ]:
inds = (df_reduced['Dose'] == 1e9) & (df_reduced['Round'] == 10)

df_4_pt3 = df_reduced.loc[inds]

In [ ]:
inds = (df_4_pt3['Virus'] == '9P31') | (df_4_pt3['Virus'] == '9P36')

df_4_pt3 = df_4_pt3.loc[inds]

In [ ]:
inds = (df_4_pt3['Receptor'] == 'Car4') | (df_4_pt3['Receptor'] == 'rsCar4') | (df_4_pt3['Receptor'] == 'hsCar4')

df_4_pt3 = df_4_pt3.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_4_pt3 = df_4_pt3.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_4_pt3)

p = figure(x_range = ['Car4',
                     'rsCar4',
                     'hsCar4'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt3['Total Brightness per Signal Area'].min(), high = df_4_pt3['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_4_pt3['Percent Positive']), max(df_4_pt3['Percent Positive']), min(df_4_pt3['Total Brightness per Signal Area']), max(df_4_pt3['Total Brightness per Signal Area'])

In [ ]:
inds = (df_reduced['Round'] == 12)

df_4_pt4 = df_reduced.loc[inds]

In [ ]:
inds = (df_4_pt4['Virus'] == '9P31') | (df_4_pt4['Virus'] == '9P36')

df_4_pt4 = df_4_pt4.loc[inds]

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt4 = df_4_pt4.astype({"Dose": str})

In [ ]:
# We can determine the mean of each of the values
df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_4_pt4

In [ ]:
for receptor in ['msCar4', 'huCar4', 'TM 118', 'TM 119', 'TM 120', 'TM 121']:
    inds = (df_4_pt4['Receptor'] == receptor)
    
    source = ColumnDataSource(df_4_pt4[inds])

    p = figure(x_range = ['9P31',
                          '9P36'],
              y_range = ['10000000.0',
                         '100000000.0',
                         '1000000000.0',
                         '10000000000.0'],
               title = receptor,
               x_axis_label = 'Virus',
               y_axis_label = 'Dose',
               width=400, 
               height=400)

    color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt4['Total Brightness per Signal Area'].min(), high = df_4_pt4['Total Brightness per Signal Area'].max())

    color_bar = ColorBar(color_mapper = color_mapper,
                         location = (0, 0),
                         ticker = BasicTicker())

    p.add_layout(color_bar, 'right')

    p.scatter(x = 'Virus', y = 'Dose', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

    p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

    p.xaxis.major_label_orientation = np.pi/2

    #p.output_backend = "svg"

    show(p)

In [ ]:
inds = (df_reduced['Dose'] == 1e9) & (df_reduced['Round'] == 12)

df_4_pt4 = df_reduced.loc[inds]

In [ ]:
inds = (df_4_pt4['Virus'] == '9P31') | (df_4_pt4['Virus'] == '9P36')

df_4_pt4 = df_4_pt4.loc[inds]

In [ ]:
inds = (df_4_pt4['Receptor'] == 'msCar4') | (df_4_pt4['Receptor'] == 'huCar4') | (df_4_pt4['Receptor'] == 'TM 118') | (df_4_pt4['Receptor'] == 'TM 119') | (df_4_pt4['Receptor'] == 'TM 120') | (df_4_pt4['Receptor'] == 'TM 121')
 
df_4_pt4 = df_4_pt4.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_4_pt4)

p = figure(x_range = ['msCar4', 
                      'huCar4', 
                      'TM 118', 
                      'TM 119', 
                      'TM 120', 
                      'TM 121'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Receptor comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt4['Total Brightness per Signal Area'].min(), high = df_4_pt4['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

---------

In [ ]:
inds = (df_reduced['DMSO (%)'] == 0.3)

df_x = df_reduced.loc[inds]

In [ ]:
inds = df_x['Signal Area'].astype(float) < df_x['Bright Field Area']*1
df_x_tempfix = df_x.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_x_grouped = df_x_tempfix.groupby(['Virus', 'Receptor', 'Dose', 'Brinzolamide (nM)']).mean().reset_index()

In [ ]:
df_x_grouped

In [ ]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_x_grouped = df_x_grouped.astype({"Brinzolamide (nM)": str})
df_x_grouped = df_x_grouped.astype({"Dose": str})

In [ ]:
for receptor in ['msCar4', 'Ly6a']:
    for virus in ['9P31', '9P36', 'PHP.eB']:
        inds = (df_x_grouped['Receptor'] == receptor) & (df_x_grouped['Virus'] == virus)

        source = ColumnDataSource(df_x_grouped[inds])

        p = figure(x_range = ['500000000.0',
                              '1000000000.0'],
                  y_range = ['0.0',
                             '1.0',
                             '100.0',
                             '300.0'],
                   title = receptor + ' and ' + virus,
                   x_axis_label = 'Virus',
                   y_axis_label = 'Dose',
                   width=300, 
                   height=400)

        color_mapper = LinearColorMapper(palette = Viridis256, low = df_x_grouped[ 'Total Brightness per Signal Area'].min(), high = df_x_grouped['Total Brightness per Signal Area'].max())

        color_bar = ColorBar(color_mapper = color_mapper,
                             location = (0, 0),
                             ticker = BasicTicker())

        p.add_layout(color_bar, 'right')

        p.scatter(x = 'Dose', y = 'Brinzolamide (nM)', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

        p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

        p.xaxis.major_label_orientation = np.pi/2

        #p.output_backend = "svg"

        show(p)

----------

In [ ]:
inds = (df_reduced['Round'] == 15) & (df_reduced['Dose'] == 1e9)
df_y = df_reduced.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
inds = (df_y_grouped['Virus'] == 'TM126') | (df_y_grouped['Virus'] == 'TM127')

df_y_grouped = df_y_grouped.loc[inds]

In [ ]:
source = ColumnDataSource(df_y_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['TM127',
                     'TM126'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_y_grouped['Total Brightness per Signal Area'].min(), high = df_y_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
# We can determine the mean of each of the values
df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
inds = (df_y_grouped['Virus'] == 'RR045') | (df_y_grouped['Virus'] == 'RR475') | \
(df_y_grouped['Virus'] == 'RD4') | (df_y_grouped['Virus'] == 'RDC16') | (df_y_grouped['Virus'] == 'MP3') | (df_y_grouped['Virus'] == 'RDM18') | \
(df_y_grouped['Virus'] == 'MP1A') | (df_y_grouped['Virus'] == 'RDM5') | (df_y_grouped['Virus'] == 'RD2') | (df_y_grouped['Virus'] == 'RDC9') | \
(df_y_grouped['Virus'] == 'RM174') | (df_y_grouped['Virus'] == 'RD5') | (df_y_grouped['Virus'] == 'MR2') | (df_y_grouped['Virus'] == 'RDC8')| \
(df_y_grouped['Virus'] == 'MP13') | (df_y_grouped['Virus'] == 'RM222')

df_y_grouped = df_y_grouped.loc[inds]

In [ ]:
source = ColumnDataSource(df_y_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['RR045',
                     'RR475',
                     'RD4',
                     'RDC16',
                     'MP3',
                     'RDM18',
                     'MP1A',
                     'RDM5',
                     'RD2',
                     'RDC9',
                     'RM174',
                     'RD5',
                     'MR2',
                     'RDC8',
                     'MP13',
                     'RM222'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=700)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_y_grouped['Total Brightness per Signal Area'].min(), high = df_y_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

----------------

In [ ]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Round'] == 13) & (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf5
df_3c = df_reduced.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_3c = df_3c.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
source = ColumnDataSource(df_3c)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      '9P08',
                      '9P16',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F',
                      '9P13',
                      '9P33',
                      '9P39'],
          y_range = ['Ly6S',
                     'Lynx1',
                     'Lypd5',
                     'Ly6h',
                     'Ly6e', 
                     'Ly6c1',
                     'Ly6a'],
           title = 'Receptor screen of Ly6 family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_3c['Total Brightness per Signal Area'].min(), high = df_3c['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

--------------

In [ ]:
inds = (df_reduced['Round'] == 16) & (df_reduced['Dose'] == 1e9)
df_z = df_reduced.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_z_grouped = df_z.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_z_grouped['Virus'].unique()


In [ ]:
source = ColumnDataSource(df_z_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['TM127', 'TM126', '9P36', '9P31', 'PHP.eB'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=800)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_z_grouped['Total Brightness per Signal Area'].min(), high = df_z_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
min(df_z_grouped['Percent Positive']), max(df_z_grouped['Percent Positive']), min(df_z_grouped['Total Brightness per Signal Area']), max(df_z_grouped['Total Brightness per Signal Area'])

-----------------

In [ ]:
inds = (df_reduced['Round'] == 17) & (df_reduced['Dose'] == 1e9)
df_a = df_reduced.loc[inds]

In [ ]:
# We can determine the mean of each of the values
df_a_grouped = df_a.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

In [ ]:
df_a_grouped['Virus'].unique(), df_a_grouped['Receptor'].unique() 

In [ ]:
source = ColumnDataSource(df_a_grouped)

p = figure(x_range = ['None',
                      'Ly6a',
                      'Marmoset Ly6E',
                      'Marmoset LyPD5',
                     'Marmoset Lynx1'],
          y_range = ['CAP-B22','CAP-B10', 'PHP.eB'],
           title = 'Marmoset Receptor Comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_a_grouped['Total Brightness per Signal Area'].min(), high = df_a_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
source = ColumnDataSource(df_a_grouped)

p = figure(x_range = ['None',
                      'msCar4', 
                      'msCar4 L71A',
                      'msCar4 L71E'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Mouse Car4 Mutants',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=500, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_a_grouped['Total Brightness per Signal Area'].min(), high = df_a_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
source = ColumnDataSource(df_a_grouped)

p = figure(x_range = ['None',
                      'Ly6c1', 
                      'Ly6c1 E31R', 
                      'Ly6c1 Q56A',
                      'Ly6c1 S94E', 
                      'Ly6c1 triple mutant'],
          y_range = ['PHP.C2',
                     'PHP.C1'],
           title = 'Ly6c1 Mutants',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=600, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_a_grouped['Total Brightness per Signal Area'].min(), high = df_a_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [ ]:
source = ColumnDataSource(df_a_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'msCar4 Q63G', 
                      'rhCar4', 
                      'rhCar4 G63Q'],
          y_range = ['TTD-008',
                     'TTD-004',
                     'TTD-001',
                     '9P36',
                     '9P31'],
           title = 'Mouse and Rhesus Comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=600, 
           height=500)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_a_grouped['Total Brightness per Signal Area'].min(), high = df_a_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)